In [133]:
import os, re
import nltk
nltk.download('punkt')
from string import punctuation
import numpy as np
from collections import Counter
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [134]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_500.txt

--2019-11-22 13:39:36--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123167 (120K) [text/plain]
Saving to: ‘sents_with_mistakes.txt.1’

sents_with_mistakes 100%[===================>] 120.28K  --.-KB/s    in 0.01s   

2019-11-22 13:39:37 (9.83 MB/s) - ‘sents_with_mistakes.txt.1’ saved [123167/123167]

--2019-11-22 13:39:39--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent,

In [0]:
def normalize(text):
    
    normalized_text = [(word.strip(punctuation)) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [0]:
corpus = []
for text in open('corpus_500.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [0]:
vocab = set()

for sent in corpus:
    vocab.update(sent)

In [0]:
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [0]:
N = sum(WORDS.values())
def P(word, N=N): 
    return WORDS[word] / N 

In [0]:
def correction(word): 
    return max(candidates(word), key=P)

def candidates(word): 
  cands = set()
  cands2 = set()
  if known2([word])!= set():
    cands.update(known2([word]))
  if  known2(edits1(word)) != set():
    cands.update(known2(edits1(word)))
  if known(edits2(word)) != set():
    cands2.update(known(edits2(word)))
  if known2(edits2(word)) != set():
    cands2.update(known(edits2(word)))
  return (known([word]) or cands or cands2 or [word] )
    #Генерируем кандидатов на исправление. Приоритет следующий: 1) есть ли это слово в списке существующих слов из корпуса 2) есть ли оно в объединенном списке кандидатов, включающем:
    #вхождения слова в словаре слов с удаленным символом(ССУС), а также вхождение слов с удаленным символом в ССУС 3) вхождение слов с 2-мя удаленными символами  в список cуществующих слов или ССУС
    #4) возвращаем слово, если ничего не нашлось'". Приоритет обусловлен предположением о том, что отсутствие ошибок более вероятно чем одна ошибка, а одна ошибка более вероятна, чем две и т.д.


def known(words): 
    return set(w for w in words if (w in vocab))

def known2 (words):
  words2 = set()
  for w in words:
    if w in vocab1.keys():
        words2.update(vocab1[w])
  return words2
   #Выбираем слова, которые есть в ССУС

def edits1(word):
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

def edits2(word): 
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

In [0]:
vocab1 = dict()
for word in vocab:
  for edited_word in edits1(word):
    if edited_word not in vocab1.keys():
      vocab1[edited_word] = {word}
    else:
      vocab1[edited_word].add(word)
#Создаем ССУС

In [12]:
correction('ооочень')

'очень'

Спеллчекер готов, далее проверим его качество

In [0]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [0]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [0]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0
cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

In [152]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)


0.6398601398601399
0.24405218726016883
0.3009073159526818


Триграммная модель

Разбиваем на триграммы корпус новостных текстов

In [0]:
corpus_news = [['<start>', '<start>'] + sent + ['<end>'] for sent in corpus]

def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams


unigrams = Counter()
bigrams = Counter()
trigrams = Counter()
mistakes = []

for sentence in corpus_news:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence, 2))
    trigrams.update(ngrammer(sentence, 3))

Также разбиваем на триграммы предложения для проверки, т.к. будут анализироваться триграммы (два слова + данное слово для исправления) именно в этих предложениях и сравниваться с триграммами в корпусе новостных текстов. При наличии аналогичных триграммов в корпусе, их частотность будет влиять на вероятность выбора того или иного слова-кандидата.  

In [0]:
def sent_tokenizer(text):
  corpus = []
  for lines in text:
    sents = lines.split('\n')
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents
  return [['<start>', '<start>'] + sent + ['<end>'] for sent in corpus]
#функция разбиения на предложения по строкам, разбиение на слова внутри предложений + добавление '<start>', '<start>' и '<end>' для триграммной модели

corpus_bad = sent_tokenizer(bad)
corpus_true = sent_tokenizer(true)

In [0]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

def correction2(dictionary): 
    return max(dictionary, key = dictionary.get)
#меняем функцию выбора кандидатов так, что на вход подается словарь, где слова - ключи, а вероятности с учетом триграммов - значения.

for j in range(len(corpus_bad)):
  for i in range(len(corpus_bad[j])):
    if corpus_bad[j][i+2] == '<end>':
      break
    current_trigram = corpus_bad[j][i] + ' ' + corpus_bad[j][i+1] + ' ' + corpus_bad[j][i+2]
    current_bigram = corpus_bad[j][i] + ' ' + corpus_bad[j][i+1]
    candid_dict = dict()
    for candidate in candidates(corpus_bad[j][i+2]):
      possible_trigram = corpus_bad[j][i] + ' ' + corpus_bad[j][i+1] + ' ' + candidate
      if possible_trigram in trigrams:
        Prob = WORDS[candidate] / N + (1 + (bigrams[current_bigram]/trigrams[possible_trigram]))
        candid_dict[candidate] = Prob
      else:
        Prob = WORDS[candidate] / N
        candid_dict[candidate] = Prob
#если биграмм + слово-кандидат есть в списке триграммов, то это меняет вероятность. Если нет, то вероятность вычисляется как и раньше.
    predicted = correction2(candid_dict)
    total+=1
    if predicted == corpus_true[j][i+2]:
      correct += 1
    if corpus_true[j][i+2] == corpus_bad[j][i+2]:
      total_correct += 1
      if corpus_true[j][i+2]  !=  predicted:
        correct_broken += 1
    else:
      total_mistaken += 1
      if corpus_true[j][i+2] == predicted:
        mistaken_fixed += 1

In [156]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.7837162837162838
0.24405218726016883
0.1355231422992994


При использовании триграммной модели число случаев, когда спеллчекер неверно исправляет правильные слова, уменьшилось более, чем в два раза. Число верно исправленных ошибок при этом не изменилось. То, что исправляется всего лишь 24%, ошибок можно объяснить тем, что корпус маловат и нерепрезентативен для составления полноценного словаря. Если бы тестовые предложения были бы взяты из новостей, то возможно и процент верно найденных ошибок был бы выше.